In [107]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException, TimeoutException

from bs4 import BeautifulSoup
import io
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC
import time
import sys
import re
import numpy as np
import os
import glob
import shutil
import json

In [108]:
!pwd

/home/mark/lambda/pill_identifier


In [109]:
output_dir = "./csresults"
input_dir = "./json/"

In [110]:
class ititle_contains(object):
    """ An expectation for checking that the title contains a case-sensitive
    substring. title is the fragment of title expected
    returns True when the title matches, False otherwise
    """
    def __init__(self, title):
        self.title = title

    def __call__(self, driver):
        return self.title.lower() in driver.title.lower()


In [122]:
#todo find the picture first, then link. Most of not found were because picture was with second link and not the first
class drugscom:
    def __init__(self):
        self.wurl = 'https://www.drugs.com/pill_identification.html'
        self.base = "https://www.drugs.com"
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('--no-sandbox')
        self.driver = webdriver.chrome.webdriver.WebDriver(
            '/usr/local/bin/chromedriver', options=chrome_options)
        self.wait = WebDriverWait(self.driver, 10)
        self.driver.set_window_size(850, 1600)
        self.shapes = [
            'Round', 'Capsole', 'Oval', 'Egg', 'Barrel',
            'Rectangle', '3 Sided', '4 Sided', '5 Sided', '6 Sided',
            '7 sided', '8 sided', 'U Shaped', 'Figure 8', 'Heart',
            'Kidney', 'Gear', 'Character'
        ]
        self.shapes_classes = [
            'circle', 'capsole', 'oval', 'egg', 'barrel',
            'rectangle', 'triangle', 'square', 'pentagon', 'hexagon',
            'heptagon', 'octagon', 'U Shaped', 'Figure 8', 'Heart',
            'kidney', 'gear', 'character'
        ]
        self.colors_classes = [None, 'White', None,
                       'Beige', 'Black', 'Blue',
                       'Brown', 'Clear', 'Gold',
                       'Gray', 'Green', 'Maroon'
                       'Orange', 'Peach', 'Pink',
                       'Purple', 'Red', 'Tan',
                       'White', 'Yellow',
                       'Beige & Red', 'Black & Green',
                       'Black & Teal', 'Black & Yellow',
                       'Blue & Brown', 'Blue & Grey'
                       'Blue & Orange', 'Blue & Peach',
                       'Blue & Pink', 'Blue & White',
                       'Blue & White Specks', 'Blue & Yellow',
                       'Brown & Clear', 'Brown & Orange',
                       'Brown & Peach', 'Brown & Red',
                       'Brown & White', 'Brown & Yellow',
                       'Clear & Green', 'Dark & Light Green',
                       'Gold & White', 'Grey & Peach',
                       'Grey & Pink', 'Grey & Red',
                       'Grey & White', 'Grey & Yellow',
                       'Green & Orange', 'Green & Peach',
                       'Green & Pink', 'Green & Purple',
                       'Green & Turquoise', 'Green & White',
                       'Green & Yellow', 'Lavender & White',
                       'Maroon & Pink', 'Orange & Turquoise',
                       'Orange & White', 'Orange & Yellow',
                       'Peach & Purple', 'Peach & Red',
                       'Peach & White', 'Pink & Purple',
                       'Pink & Red Specks', 'Pink & Turquoise',
                       'Pink & White', 'Pink & Yellow',
                       'Red & Turquoise', 'Red & White',
                       'Red & Yellow', 'Tan & White',
                       'Turquoise & White', 'Turquuise & Yellow',
                       'White & Blue Specks', 'White & Red Specks',
                       'White & Yellow', 'Yellow & Grey',
                       'Yellow & White'
                       ]
        self.colors = [
            'Beige', 'Black', 'Blue',
            'Brown', 'Clear', 'Gold',
            'Gray', 'Green', 'Maroon'
            'Orange', 'Peach', 'Pink',
            'Purple', 'Red', 'Tan',
            'White', 'Yellow',
            'Beige & Red', 'Black & Green',
            'Black & Teal', 'Black & Yellow',
            'Blue & Brown', 'Blue & Grey'
            'Blue & Orange', 'Blue & Peach',
            'Blue & Pink', 'Blue & White',
            'Blue & White Specks', 'Blue & Yellow',
            'Brown & Clear', 'Brown & Orange',
            'Brown & Peach', 'Brown & Red',
            'Brown & White', 'Brown & Yellow',
            'Clear & Green', 'Dark & Light Green',
            'Gold & White', 'Grey & Peach',
            'Grey & Pink', 'Grey & Red',
            'Grey & White', 'Grey & Yellow',
            'Green & Orange', 'Green & Peach',
            'Green & Pink', 'Green & Purple',
            'Green & Turquoise', 'Green & White',
            'Green & Yellow', 'Lavender & White',
            'Maroon & Pink', 'Orange & Turquoise',
            'Orange & White', 'Orange & Yellow',
            'Peach & Purple', 'Peach & Red',
            'Peach & White', 'Pink & Purple',
            'Pink & Red Specks', 'Pink & Turquoise',
            'Pink & White', 'Pink & Yellow',
            'Red & Turquoise', 'Red & White',
            'Red & Yellow', 'Tan & White',
            'Turquoise & White', 'Turquuise & Yellow',
            'White & Blue Specks', 'White & Red Specks',
            'White & Yellow', 'Yellow & Grey',
            'Yellow & White'
        ]

        self.results = []

    def get_data(self, pmprint):
        self.driver.get(self.wurl)
        WebDriverWait(self.driver, 100).until(EC.title_contains(
            "Pill Identifier (Pill Finder) - Drugs.com"))
        elem = self.wait.until(
            EC.element_to_be_clickable((By.LINK_TEXT, 'Accept')))
        elem.click()
        mprint = pmprint
#         elem = self.wait.until(EC.element_to_be_clickable((By.XPATH, "//a[@href='/imprints.php']")))
        elem = self.wait.until(EC.element_to_be_clickable(
            (By.CSS_SELECTOR, "#livesearch-imprint")))
        elem.click()
        elem.clear()
        elem.send_keys(mprint)
        elem.send_keys(Keys.RETURN)
        elem = self.wait.until(EC.element_to_be_clickable(
            (By.XPATH, "//input[@type='submit']")))
        soup = BeautifulSoup(self.driver.page_source, 'html.parser') 
        i = 0
        for elem in soup(text='Pill Imprint:'):
            a = elem.parent.find_next('a')
#             print(f"img: {a['href']} MPRINT: {a.text}")
            mprint = a.text
            mprint = ' '.join(mprint.split()) # remove repeated internal spaces
            self.driver.get(self.base + a['href'])
            WebDriverWait(self.driver, 100).until(ititle_contains(mprint))
#             print(a.text, ' title')
            isoup = BeautifulSoup(self.driver.page_source, 'html.parser')
            #<dt class="pid-item-title pid-item-inline">Color:</dt>
            colors = isoup.find_all('dt', string='Color:')
#             print('colors', colors)
            shapes = isoup.find_all('dt', string='Shape:')
            imgs = isoup.find_all('img')
#             print('imgs len',len(imgs))
            for img in imgs:
                #                   print('img', img)
                try:
                    s = img['src']
#                       print('s',s)
                    if s[0:14] == '/images/pills/':
                        #                          print('found img', s, ' mprint ', mprint)
                        #                         self.img = base + s
                        self.results.append(
                            {'mprint': mprint, 'img': self.base + s, 'color': colors[i].next_sibling.text, 'shape': shapes[i].next_sibling.text})
                        break
                except:
                    pass
            if mprint.lower() == pmprint.lower(): # works because drugs.com puts matching mprint first
                break
            i += i
        return self.results

    def reset(self):
        self.results = []


In [112]:
pd.set_option('display.max_columns', 500)

In [113]:
url = 'pillbox_201805.tab'
df_ = pd.read_csv(url,sep='\t')

In [114]:
df = df_[(df_.SPLIMPRINT.notnull()) & (df_.image_id.notnull()) & (df_.image_id != 'no_product_image')]

In [115]:
df.head()

,id,spl_id,SETID,spp,NDC9,PRODUCT_CODE,EQUAL_PRODUCT_CODE,author,SPLIMPRINT,SPLCOLOR,SPLSHAPE,SPLSIZE,SPLSCORE,DEA_SCHEDULE_CODE,INGREDIENTS,SPL_INACTIVE_ING,RXCUI,RXTTY,RXSTRING,image_id,IMAGE_SOURCE,HAS_IMAGE,FROM_SIS,NO_RXCUI
50,4710,NaN,8fed703a-9de2-4c76-a783-fcf004f9b890,8fed703a-9de2-4c76-a783-fcf004f9b890-50844-175-0,508440175,50844-175,NaN,"L.N.K. International, Inc.",44;175,C48325,C48345,17.0,1,NaN,ACETAMINOPHEN[ACETAMINOPHEN],HYPROMELLOSES / POVIDONE / STEARIC ACID / CAST...,198440.0,SCD,Acetaminophen 500 MG Oral Tablet,50844-0175-10_NLMIMAGE10_5135A8AD,NLMIMAGE10,1,NaN,NaN
58,61985,NaN,c35d5dcc-8ee1-49da-993d-65c0195db918,c35d5dcc-8ee1-49da-993d-65c0195db918-70518-0901-0,705180901,70518-0901,0093-7206,REMEDYREPACK INC.,9;3;7206,C48330,C48348,7.0,2,NaN,MIRTAZAPINE[MIRTAZAPINE],"SILICON DIOXIDE / STARCH, CORN / HYPROMELLOSE ...",311725.0,SCD,Mirtazapine 15 MG Oral Tablet,00093720656,SPL,1,1.0,NaN
68,11648,NaN,2fb7763f-6ce4-43b9-b3be-90a66315828b,2fb7763f-6ce4-43b9-b3be-90a66315828b-60429-566-0,604290566,60429-566,0591-0744,"Golden State Medical Supply, Inc.",WATSON;744;1,C48325,C48338,9.0,2,C48677,ESTAZOLAM[ESTAZOLAM],DOCUSATE SODIUM / LACTOSE MONOHYDRATE / MAGNES...,NaN,NaN,NaN,604290566,VA2,1,1.0,NaN
71,47756,NaN,662f11ca-cf1e-41fd-9b2a-11a23d1f6290,662f11ca-cf1e-41fd-9b2a-11a23d1f6290-0781-1034-0,007811034,0781-1034,NaN,Sandoz Inc,GG55;5,C48327,C48348,8.0,1,NaN,TRIFLUOPERAZINE HYDROCHLORIDE[TRIFLUOPERAZINE],D&C RED NO. 30 / FD&C BLUE NO. 2 / HYDROXYPROP...,198325.0,SCD,Trifluoperazine 5 MG Oral Tablet,007811034,VA2,1,NaN,NaN
79,33477,NaN,70be7d33-bc8b-4f48-84c6-a8e3d1476e97,70be7d33-bc8b-4f48-84c6-a8e3d1476e97-59762-0145-0,597620145,59762-0145,NaN,Greenstone LLC,FELDENE;PFIZER;323,C48326,C48336,18.0,1,NaN,PIROXICAM[PIROXICAM],"FD&C BLUE NO. 1 / FD&C RED NO. 3 / LACTOSE, UN...",198108.0,SCD,Piroxicam 20 MG Oral Capsule,59762-0145-01_NLMIMAGE10_333E99F4,RXIMAGE30,1,NaN,NaN


In [116]:
regex = r"([\w\d]+)"

In [117]:
js_names = glob.glob(os.path.join(input_dir, '*.json')) 

In [123]:
print(len(js_names))

7001


In [124]:
d = drugscom()

In [125]:
dicta = []
i = 0
for fp in js_names:
    image_id = fp[7:-9]
    d.reset()
    j = json.load(open(fp, 'r'))
    mprint = j['description']
    mprint = ' '.join(mprint.split()) # remove repeated internal spaces
    dict = d.get_data(mprint)
    if len(dict) == 0:
        print(f"mprint {mprint} in {image_id} not found")
        i += 1
        continue
    dict = dict[0]
#     print(mprint, d.get_data(mprint), fp[9:-9])
    try: 
        row = df[df.image_id == image_id][['SPLCOLOR','SPLSHAPE']].iloc[0]
#     print('|' + image_id + '|', row['SPLCOLOR'], row['SPLSHAPE'], type(row))
        dicta.append({'image_id': image_id, 'color': dict['color'],'SPLCOLOR': row['SPLCOLOR'], 'shape': dict['shape'],'SPLSHAPE': row['SPLSHAPE']  })
    except:
        print(f"row {mprint} {image_id} not found")
    i += 1
    if i % 100 == 0:
        print('i',i,end=' ')
#     break

mprint IBU 200 not found
mprint C 12 not found
i 100 mprint Andrx 696 120 mg not found
mprint S4 not found
i 200 mprint PREMARIN 09 not found
mprint f 657 5 mg not found
i 300 mprint PREMARIN 045 not found
mprint Andrx 697 180 mg not found
mprint 791 CTI 141 not found
mprint 792 CTI 142 not found
mprint 20mg POT20 not found
mprint UPPER tcl334 LOWER PLAIN not found
mprint Andrx 699 300 mg not found
mprint Y not found
mprint W 0625 5 not found
i 500 mprint ACD 324 not found
mprint Plain C020 not found
mprint 500125 AMC not found
mprint R127 not found
i 600 mprint MACRODANTIN 100mg 52427 286 not found
i 700 mprint 103 Caraco not found
mprint PENTASA 250 mg S429 250 mg not found
mprint WW 3189 not found
i 800 mprint PREMARIN 0625 not found
mprint PENTASA 500 mg S429 500 mg not found
mprint C83 not found
mprint Pfizer CDT 254 not found
i 900 mprint PREMPRO 0625 25 not found
mprint WW 947 not found
mprint ETHEX 309 5 not found
mprint PLAIN C010 not found
i 1000 mprint E7 not found
mprint A2

IndexError: single positional indexer is out-of-bounds

In [5]:
d = drugscom()
d.get_data("C 10")
d.get_data("LILLY 3240 30 mg")
d.get_data("LILLY 4117")

[{'mprint': 'C 10',
  'img': 'https://www.drugs.com/images/pills/nlm/200000010.jpg'},
 {'mprint': 'Lilly 3240 30 mg',
  'img': 'https://www.drugs.com/images/pills/nlm/000023240.jpg'},
 {'mprint': 'LILLY 4117',
  'img': 'https://www.drugs.com/images/pills/nlm/000024117.jpg'}]

In [178]:
d.get_data("10 C")

[{'mprint': 'C 10',
  'img': 'https://www.drugs.com/images/pills/nlm/200000010.jpg'},
 {'mprint': 'Lilly 3240 30 mg',
  'img': 'https://www.drugs.com/images/pills/nlm/000023240.jpg'},
 {'mprint': 'LILLY 4117',
  'img': 'https://www.drugs.com/images/pills/nlm/000024117.jpg'},
 {'mprint': 'PERCOCET 10/325',
  'img': 'https://www.drugs.com/images/pills/nlm/634810629.jpg'},
 {'mprint': 'BAC 10 832',
  'img': 'https://www.drugs.com/images/pills/nlm/008321024.jpg'},
 {'mprint': 'M EC 10',
  'img': 'https://www.drugs.com/images/pills/nlm/200000822.jpg'}]

In [179]:
d.reset()
d.get_data("4117 LILLY")

[{'mprint': 'LILLY 4117',
  'img': 'https://www.drugs.com/images/pills/nlm/000024117.jpg'}]

In [129]:
len(dicta)

2053

In [176]:
dfd = pd.DataFrame(dicta) #todo never name a DataFrame column "shape" again

In [177]:
dfd.head()

,SPLCOLOR,SPLSHAPE,color,image_id,shape
0,C48328,C48348,Pink,433530866,Round
1,C48331,C48345,Peach,009551046,Elliptical / Oval
2,C48325,C48348,White,000780017,Round
3,C48329;C48327,C48336,Green & Purple,00093316006,Capsule-shape
4,C48325,C48348,White,510790810,Round


In [132]:
with open('color_shape.csv', 'wt') as f:
    dfd.to_csv(f)

In [133]:
!pwd

/home/mark/lambda/pill_identifier


In [186]:
ushapes = dfd['shape'].unique()
sdict = {}
for splshape in dfd.SPLSHAPE.unique():
    rd = {}
    for shape in ushapes:
        c = len(dfd[(dfd.SPLSHAPE == splshape) & (dfd['shape'] == shape)])
        if c > 0:
            rd[shape] = c
    sdict[splshape] = rd

In [179]:
ucolors = dfd.color.unique()
cdict = {}
for splcolor in dfd.SPLCOLOR.unique():
    rd = {}
    for color in ucolors:
        c = len(dfd[(dfd.SPLCOLOR == splcolor) & (dfd.color == color)])
        if c > 0:
            rd[color] = c
    cdict[splcolor] = rd

In [141]:
from functools import reduce

In [187]:
sd = {}
for cd in sdict.keys():
    d = sdict[cd]
    da = reduce((lambda m, v: [d[v],v] if d[v] > m[0] else m), d.keys(), [0,'x'])
    sd[cd] = da[1]

In [188]:
sd

{'C48348': 'Round',
 'C48345': 'Elliptical / Oval',
 'C48336': 'Capsule-shape',
 'C48346': 'Five-sided',
 'C48340': 'Heart-shape',
 'C48350': 'Four-sided',
 'C48338': 'Four-sided',
 'C48351': 'Elliptical / Oval',
 'C48347': 'Rectangle',
 'C48353': 'Three-sided',
 'C48352': 'Four-sided',
 'C48343': 'Six-sided',
 'C48349': 'U-shape',
 'C48335': 'Capsule-shape',
 'C48344': 'Eight-sided'}

In [146]:
fd = {}
for cd in cdict.keys():
    d = cdict[cd]
    da = reduce((lambda m, v: [d[v],v] if d[v] > m[0] else m), d.keys(), [0,'x'])
    fd[cd] = da[1]

In [189]:
ssd = {}
for key in sorted(sd.keys()):
    ssd[key] = sd[key]

In [190]:
ssd

{'C48335': 'Capsule-shape',
 'C48336': 'Capsule-shape',
 'C48338': 'Four-sided',
 'C48340': 'Heart-shape',
 'C48343': 'Six-sided',
 'C48344': 'Eight-sided',
 'C48345': 'Elliptical / Oval',
 'C48346': 'Five-sided',
 'C48347': 'Rectangle',
 'C48348': 'Round',
 'C48349': 'U-shape',
 'C48350': 'Four-sided',
 'C48351': 'Elliptical / Oval',
 'C48352': 'Four-sided',
 'C48353': 'Three-sided'}

In [170]:
sfd = {}
for key in sorted(fd.keys()):
    sfd[key] = fd[key]

In [171]:
for key in fd.keys():
    sk = key.split(';')
    sd = sfd[key].split(' & ')
    if len(sd) == 1:
        sd = sfd[key].split(' / ')
    subkeys = (len(sk) == len(sd)) & (len(sd) > 1)
    r = subkeys
    if subkeys:
        for i in range(len(sd)):
            try:
                if sfd[sk[i]] not in sd:
                    r = False
                    break
            except:
                r = False
                break;
    if r:
        if r & subkeys:
            print('deleting key ', key)
        del sfd[key]
            
            

deleting key  C48329;C48327
deleting key  C48329;C48328
deleting key  C48330;C48323
deleting key  C48325;C48333
deleting key  C48333;C48325
deleting key  C48329;C48325
deleting key  C48333;C48330
deleting key  C48326;C48325
deleting key  C48330;C48325
deleting key  C48328;C48325
deleting key  C48325;C48328
deleting key  C48331;C48325
deleting key  C48325;C48327
deleting key  C48329;C48330
deleting key  C48325;C48331
deleting key  C48325;C48330
deleting key  C48323;C48329
deleting key  C48328;C48333
deleting key  C48333;C48331
deleting key  C48324;C48325
deleting key  C48331;C48332
deleting key  C48333;C48329
deleting key  C48331;C48330
deleting key  C48329;C48324
deleting key  C48325;C48324
deleting key  C48324;C48331
deleting key  C48328;C48329
deleting key  C48327;C48328
deleting key  C48327;C48325
deleting key  C48324;C48330
deleting key  C48331;C48329
deleting key  C48333;C48324
deleting key  C48330;C48326


In [173]:
for key in fd.keys():
    if key in sfd:
        if len(key.split(';')) > 1: 
            del sfd[key]

In [174]:
sfd

{'C48323': 'Black',
 'C48324': 'Gray',
 'C48325': 'White',
 'C48326': 'Red',
 'C48327': 'Purple',
 'C48328': 'Pink',
 'C48329': 'Green',
 'C48330': 'Yellow',
 'C48331': 'Orange',
 'C48332': 'Brown',
 'C48333': 'Blue',
 'C48334': 'Blue'}